In [ ]:
import os 
import pandas as pd
import requests

In [ ]:
# recuperer les codes postaux du rhône
df_cp = pd.read_csv('adresses-69.csv',  sep=';')
code_postaux = sorted(df_cp["code_postal"].unique().tolist())

In [ ]:
colonnes = ",".join([
    "configuration_installation_chauffage_n1","conso_chauffage_installation_chauffage_n1","type_generateur_n1_ecs_n1","numero_voie_ban","score_ban","surface_habitable_immeuble",
    "conso_auxiliaires_ep","deperditions_murs","cout_eclairage","conso_auxiliaires_ef","statut_geocodage","ventilation_posterieure_2012","cout_chauffage","conso_5_usages_par_m2_ep",
    "date_etablissement_dpe","conso_ecs_ef_energie_n1","conso_ecs_ef_energie_n2","emission_ges_chauffage","description_installation_chauffage_n1","conso_5_usages_par_m2_ef","cout_ecs_energie_n2",
    "conso_chauffage_ef_energie_n1","conso_chauffage_ef_energie_n2","qualite_isolation_menuiseries","cout_total_5_usages_energie_n2","date_reception_dpe","cout_total_5_usages_energie_n1","cout_ecs_energie_n1",
    "qualite_isolation_plancher_bas","modele_dpe","qualite_isolation_enveloppe","conso_chauffage_generateur_n1_installation_n1","type_energie_n1","emission_ges_eclairage","type_energie_n2","code_postal_ban","emission_ges_ecs",
    "conso_5_usages_ef_energie_n2","conso_5_usages_ef","conso_5_usages_ef_energie_n1","code_insee_ban","deperditions_planchers_bas","conso_5_usages_ep","date_fin_validite_dpe","deperditions_enveloppe","code_region_ban","volume_stockage_generateur_n1_ecs_n1",
    "surface_chauffee_installation_chauffage_n1","version_dpe","besoin_ecs","coordonnee_cartographique_x_ban","type_generateur_chauffage_principal","type_energie_principale_ecs","apport_solaire_saison_chauffe","adresse_ban","nombre_appartement","deperditions_renouvellement_air",
    "_rand","surface_habitable_desservie_par_installation_ecs_n1","production_electricite_pv_kwhep_par_an","type_installation_chauffage","nombre_niveau_logement","surface_habitable_logement","cout_ecs","type_installation_ecs_n1","emission_ges_5_usages_energie_n1",
    "emission_ges_5_usages_energie_n2","apport_interne_saison_froide","emission_ges_5_usages_par_m2","description_generateur_chauffage_n1_installation_n1","qualite_isolation_plancher_haut_comble_perdu","apport_interne_saison_chauffe","apport_solaire_saison_froide",
    "type_generateur_n1_installation_n1","nombre_logements_desservis_par_installation_ecs_n1","complement_adresse_logement","cout_auxiliaires","type_emetteur_installation_chauffage_n1","besoin_chauffage","configuration_installation_ecs_n1","description_installation_ecs_n1",
    "classe_inertie_batiment","deperditions_ponts_thermiques","type_generateur_chauffage_principal_ecs","emission_ges_refroidissement","hauteur_sous_plafond","conso_chauffage_ef","nom_commune_ban","annee_construction","_geopoint","date_visite_diagnostiqueur","type_batiment",
    "periode_construction","type_installation_ecs","conso_ecs_ep","conso_ecs_ef","emission_ges_5_usages","date_derniere_modification_dpe","etiquette_ges","identifiant_ban","deperditions_baies_vitrees","type_energie_generateur_n1_ecs_n1","ubat_w_par_m2_k","numero_etage_appartement",
    "nom_commune_brut","conso_ef_installation_ecs_n1","etiquette_dpe","description_generateur_n1_ecs_n1","code_departement_ban","type_installation_chauffage_n1","methode_application_dpe","adresse_brut","cout_total_5_usages","categorie_enr","conso_refroidissement_ef","conso_eclairage_ef","deperditions_planchers_hauts",
    "zone_climatique","conso_ef_generateur_n1_ecs_n1","emission_ges_ecs_energie_n1","emission_ges_ecs_energie_n2","cout_refroidissement","conso_chauffage_ep","conso_eclairage_ep","usage_generateur_n1_installation_n1","nom_rue_ban","qualite_isolation_murs","type_installation_solaire_n1",
    "classe_altitude","conso_refroidissement_ep","type_energie_principale_chauffage","numero_dpe","_i","besoin_refroidissement","emission_ges_chauffage_energie_n2","emission_ges_chauffage_energie_n1","cout_chauffage_energie_n2","deperditions_portes","cout_chauffage_energie_n1","coordonnee_cartographique_y_ban",
    "type_energie_generateur_n1_installation_n1","code_postal_brut","emission_ges_auxiliaires","usage_generateur_n1_ecs_n1"
])


In [ ]:
import requests
import pandas as pd
import time
import random
import math

base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
output_file = "existants_69.csv"
page_size = 1000   # 1k par page ; page*size must be <=10000 (API)
max_retries = 3
sleep_between_requests = (0.1, 0.6)  # random sleep to avoid bursts
max_workers = 8  # pour rester bien en-dessous des limites 
annees = ["2021","2022","2023","2024","2025"]

# recuperer les codes postaux du rhône
df_cp = pd.read_csv('adresses-69.csv',  sep=';')
code_postaux = sorted(df_cp["code_postal"].unique().tolist())



def request_with_retries(params):
    """GET simple avec retry/backoff; retourne response ou None"""
    backoff = 1
    for attempt in range(1, max_retries+1):
        try:
            r = requests.get(base_url, params=params, timeout=20)
            if r.status_code == 200:
                return r
            # pour 429 ou 5xx -> retry
            if r.status_code == 429 or 500 <= r.status_code < 600:
                print(f"Got {r.status_code}. retry {attempt}/{max_retries} after backoff {backoff}s")
                time.sleep(backoff + random.random()*0.5)
                backoff *= 2
                continue
            # pour 400/404 etc. -> on ne retry pas mais on le loggue
            print("REQUEST URL:", r.request.url)
            print("STATUS:", r.status_code)
            print("BODY:", r.text[:1000])  # tronque pour ne pas spam
            return r
        except requests.RequestException as e:
            print(f"Network error (attempt {attempt}): {e}. backoff {backoff}s")
            time.sleep(backoff)
            backoff *= 2
    return None

def fetch_for_cp(cp, annee=None):
    cp = str(cp).strip()
    if not cp:
        return []
    all_results = []
    page = 1

    while True:
        params = {
            "page": page,
            "size": page_size,
            "select": colonnes,
            "q": cp,
            "q_fields": "code_postal_ban"
        }
        if annee:
            # filtre sur la date de réception DPE pour découper
            params["qs"] = f"date_reception_dpe:[{annee}-01-01 TO {annee}-12-31]"

        r = request_with_retries(params)
        if r is None:
            print(f"Échec réseau total pour {cp} (année={annee}) page {page}")
            break

        if r.status_code != 200:
            # debug info (déjà imprimée dans request_with_retries)
            break

        content = r.json()
        results = content.get("results", [])
        total = content.get("total", 0)

        if not results:
            break

        all_results.extend(results)
        print(f"{cp} | année={annee or 'ALL'} | page={page} | got {len(results)} | total reported {total}")

        # pagination stop condition — safe: page*size >= total
        if (page * page_size) >= total:
            break
        page += 1

        time.sleep(random.uniform(*sleep_between_requests))

    return all_results

# boucle principale
all_rows = []
first_save = True
for cp in code_postaux:
    data = fetch_for_cp(cp)
    # si >=10000 on découpe par année
    if len(data) >= 10000:
        print(f"⚠️ {cp} a {len(data)} résultats, découpage par année...")
        data = []
        for an in annees:
            data.extend(fetch_for_cp(cp, annee=an))
            time.sleep(random.uniform(*sleep_between_requests))

    if data:
        df_temp = pd.DataFrame(data)
        df_temp.to_csv(output_file, mode="a", index=False, header=first_save)
        first_save = False
        all_rows.extend(data)

    # petite pause entre CP pour limiter le débit global
    time.sleep(random.uniform(0.4, 1.0))

print("Terminé. Total lignes récupérées:", len(all_rows))
